In [1]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch
from sklearn.metrics import classification_report

In [2]:

preprocessed_credit_risk_df = pd.read_csv('preprocessed_credit_risk.csv')

preprocessed_credit_risk_df.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_Y
0,22,59000,50.0,35000,16.02,1,0.59,3,False,False,True,False,False,True,False,False,False,True
1,24,83000,8.0,35000,8.90,1,0.42,2,False,False,True,False,False,False,False,False,False,False
2,25,137000,9.0,34800,16.77,0,0.25,2,False,False,True,False,False,False,True,False,False,True
3,24,10980,0.0,1500,7.29,0,0.14,3,False,True,False,False,False,False,False,False,False,False
4,22,80000,3.0,33950,14.54,1,0.42,4,False,False,True,False,False,True,False,False,False,True


In [3]:
# Split features and target
X = preprocessed_credit_risk_df.drop("loan_status", axis=1)
y = preprocessed_credit_risk_df["loan_status"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [4]:
# Build model function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    # Tune number of layers
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(keras.layers.Dense(
            units=hp.Int(f"units_{i}", min_value=16, max_value=128, step=16),
            activation=hp.Choice("activation", ["relu", "tanh"])
        ))
    model.add(keras.layers.Dense(1, activation="sigmoid"))  # Binary output

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [5]:
# Set up tuner
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    directory="credit_risk_tuning",
    project_name="personal_loans"
)

Reloading Tuner from credit_risk_tuning/personal_loans/tuner0.json


In [6]:
# Run search
tuner.search(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

In [7]:
# Retrieve best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print best hyperparameter values
print("Best Hyperparameters:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")


Best Hyperparameters:
num_layers: 3
units_0: 64
activation: tanh
learning_rate: 0.0003244843025329892
units_1: 48
units_2: 112


In [8]:
# Get best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate
y_pred = (best_model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
              precision    recall  f1-score   support

           0       0.86      0.97      0.91       783
           1       0.78      0.36      0.49       193

    accuracy                           0.85       976
   macro avg       0.82      0.67      0.70       976
weighted avg       0.84      0.85      0.83       976



/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:


def build_model_2(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    # Tune number of layers (2 to 5)
    for i in range(hp.Int("num_layers", 2, 5)):
        model.add(keras.layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=256, step=32),
            activation=hp.Choice("activation", ["relu", "tanh"])
        ))
        # Add dropout after each dense layer
        model.add(keras.layers.Dropout(
            hp.Float(f"dropout_{i}", min_value=0.1, max_value=0.5, step=0.1)
        ))

    # Output layer for binary classification
    model.add(keras.layers.Dense(1, activation="sigmoid"))

    # Tune optimizer
    optimizer = hp.Choice("optimizer", ["adam", "rmsprop", "nadam"])

    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy", keras.metrics.AUC(name="auc")]
    )

    return model


In [14]:
from keras_tuner import RandomSearch

tuner = RandomSearch(
    build_model_2,
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=1,
    directory='new_credit_risk_tuning',
    project_name='personal_loans_v2'
)



In [15]:

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5)

tuner.search(
    X_train, y_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)


Trial 50 Complete [00h 00m 03s]
val_accuracy: 0.8361075520515442

Best val_accuracy So Far: 0.8437899947166443
Total elapsed time: 00h 02m 53s


In [16]:
# Retrieve best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print best hyperparameter values
print("Best Hyperparameters:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")


Best Hyperparameters:
num_layers: 2
units_0: 96
activation: tanh
dropout_0: 0.2
units_1: 64
dropout_1: 0.30000000000000004
optimizer: nadam
units_2: 128
dropout_2: 0.4
units_3: 192
dropout_3: 0.4
units_4: 32
dropout_4: 0.1


In [17]:
# Get best model
best_model_2 = tuner.get_best_models(num_models=1)[0]

# Evaluate
y_pred = (best_model_2.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.86      0.97      0.91       783
           1       0.78      0.36      0.49       193

    accuracy                           0.85       976
   macro avg       0.82      0.67      0.70       976
weighted avg       0.84      0.85      0.83       976



/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 15 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
